In [20]:
import json
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import random
# import networkx as nx
import multiprocessing as mp
import csv


import time
import seaborn as sns
sns.set(color_codes=True)

import matplotlib.gridspec as gridspec
from scipy.cluster import hierarchy

import matplotlib as mpl
mpl.style.use('seaborn-talk')
%matplotlib inline

In [21]:
def tweet_extrac(tweet):
  tweet_id = tweet["id_str"] # Identificador del tweet
  date_tweet=tweet["created_at"]
  user_id = tweet["user"]["id_str"] # Identificador del usuario
  user_name = tweet["user"]["screen_name"] # Nombre del usuario
  user_location = tweet["user"]["location"]
  user_verified = tweet["user"]["verified"]
  user_followers_count = tweet["user"]["followers_count"]
  user_friends_count = tweet["user"]["friends_count"]
  user_favourites_count = tweet["user"]["favourites_count"]
  user_statuses_count = tweet["user"]["statuses_count"]
  user_created_at = tweet["user"]["created_at"]
  is_retweet=False

  tw_text=tweet["text"]
  if tweet["truncated"]:#=="True": # Truncated indica si el tweet es corto o largo; si truncado, busca extendido
      tw_text=tweet["extended_tweet"]["full_text"]
  else:
      if tweet["text"][0:2]=="RT":
          is_retweet=True
          try:
            if tweet["retweeted_status"]["truncated"]:
              tw_text=tweet["retweeted_status"]["extended_tweet"]["full_text"]
            else: 
                tw_text=tweet["retweeted_status"]["text"]
          except:
            tw_text=tweet["text"]
  is_quote_status=tweet["is_quote_status"] #True/False
  user_id_replyed=tweet['in_reply_to_user_id']#
  user_name_replyed=tweet["in_reply_to_screen_name"]
  tweet_id_replyed=tweet['in_reply_to_status_id']
  quote_count=tweet['quote_count']
  reply_count=tweet['reply_count']
  retweet_count=tweet['retweet_count']
  retweeted=tweet["retweeted"]
  tweet_url="https://twitter.com/"+user_name+"/status/"+tweet_id
  is_reply=False
  if not(tweet['in_reply_to_user_id']==None):
    is_reply=True

  return {"tweet_id":tweet_id,
          "date_tweet":date_tweet,
          "user_id":user_id,
          "user_name":user_name,
          "user_location":user_location,
          "user_verified":user_verified,
          "user_followers_count":user_followers_count,
          "user_friends_count":user_friends_count,
          "user_favourites_count":user_favourites_count,
          "user_statuses_count":user_statuses_count,
          "user_created_at":user_created_at,
          "is_retweet":is_retweet,
          "tw_text":tw_text,
          "is_quote_status":is_quote_status,
          "is_reply":is_reply,
          "user_id_replyed":user_id_replyed,
          "user_name_replyed":user_name_replyed,
          "tweet_id_replyed":tweet_id_replyed,
          "quote_count":quote_count,
          "reply_count":reply_count,
          "retweet_count":retweet_count,
          "retweeted":retweeted,
          "tweet_url":tweet_url
         }


In [22]:
tweets_datas_path = ["observatorio_genero_14.json"]

In [39]:
time_1_read=time.time()
i_th=0

for json_file in tweets_datas_path:
    with open(json_file, "r") as tweets_file:
        for line in tweets_file:
            try:
                tw_entities=[]
                tweet = json.loads(line)
                tw_entities.extend(tweet["entities"]["user_mentions"])
                if tweet["text"][0:2]=="RT":
                    try:
                        if tweet["retweeted_status"]["truncated"]:
                            tw_entities.extend(tweet["retweeted_status"]["extended_tweet"]["entities"]["user_mentions"])
                        else:
                            tw_entities.extend(tweet["retweeted_status"]["entities"]["user_mentions"])
                    except:
                        tw_entities.extend(tweet["entities"]["user_mentions"])
                extracted=tweet_extrac(tweet) #extracts_twts.append(extracted)
                mentions_info=[tweet["id_str"],
                               tweet["user"]["screen_name"],
                               list(set([ent["id_str"] for ent in tw_entities])),
                               list(set([ent["screen_name"] for ent in tw_entities]))]
#                 out_mentions_cands=[tweet["id_str"]]
#                 out_mentions_cands.extend([cand_id in mentions_info[2] for cand_id in ids_cands])
#                 with open('cands_mentions.csv', 'a', newline='') as cnd_mnts:
#                     writer = csv.writer(cnd_mnts)
#                     writer.writerow(out_mentions_cands)#tweet["id_str"],
                with open('mentions.csv', 'a', newline='') as ments_gral:
                    writer = csv.writer(ments_gral)
                    writer.writerow(mentions_info)
                with open('extracted_dict.csv', 'a', newline='') as ext_dict:
                    writer = csv.writer(ext_dict)
                    writer.writerow(list(extracted.values()))
                i_th+=1
            except:
                continue
print(json_file,"completed. Total time:", str(time.time()-time_1_read),"seconds. Tweets:",i_th)
# print(str(time.time()-time_1_read),"seconds; ",str(len(tweet_complete)),"registros")

observatorio_genero_14.json completed. Total time: 210.06323504447937 seconds. Tweets: 47905


In [117]:
extracted.keys()

dict_keys(['tweet_id', 'date_tweet', 'user_id', 'user_name', 'user_location', 'user_verified', 'user_followers_count', 'user_friends_count', 'user_favourites_count', 'user_statuses_count', 'user_created_at', 'is_retweet', 'tw_text', 'is_quote_status', 'is_reply', 'user_id_replyed', 'user_name_replyed', 'tweet_id_replyed', 'quote_count', 'reply_count', 'retweet_count', 'retweeted', 'tweet_url'])

# Explorar un tuit

In [35]:
limit=100
tw_completo=[]
with open("observatorio_genero_14.json", "r") as tweets_file:
    count=0
    for line in tweets_file:
        tweet=json.loads(line)
        tw_completo.append(tweet)
        if count>limit:
            break
        count+=1


In [102]:
tweet=tw_completo[6]
# print(json.dumps(tweet, indent=4, sort_keys=True))

In [120]:
print(tweet_extrac(tweet)["tw_text"])

📽💞 Marxismo, socialismo, obreras y huelgas: sí, todas estas son claves para entender cómo nació el #DiaDeLaMujer
#8M #8M2020 #8marzo2020
https://t.co/Z7JKuZzKEl


In [104]:
tw_entities_mentions=[]
# tweet = tw_completo[82]#json.loads(line)
tw_entities_mentions.extend(tweet["entities"]["user_mentions"])
if tweet["text"][0:2]=="RT":
    try:
        if tweet["retweeted_status"]["truncated"]:
            tw_entities_mentions.extend(tweet["retweeted_status"]["extended_tweet"]["entities"]["user_mentions"])
        else:
            tw_entities_mentions.extend(tweet["retweeted_status"]["entities"]["user_mentions"])
    except:
        tw_entities_mentions.extend(tweet["entities"]["user_mentions"])
extracted=tweet_extrac(tweet) #extracts_twts.append(extracted)
names_mentions=list(set([ent["screen_name"] for ent in tw_entities_mentions]))
mentions_info=[tweet["id_str"],
               tweet["user"]["screen_name"],
               tweet["user"]["id_str"],
#                list(set([ent["id_str"] for ent in tw_entities])),
               
# out_mentions_cands=[tweet["id_str"]]
# out_mentions_cands.extend([cand_id in mentions_info[2] for cand_id in ids_cands])


In [105]:
tw_entities_hashtags=[]
# tweet = tw_completo[82]#json.loads(line)
tw_entities_hashtags.extend(tweet["entities"]["hashtags"])
if tweet["text"][0:2]=="RT":
    try:
        if tweet["retweeted_status"]["truncated"]:
            tw_entities_hashtags.extend(tweet["retweeted_status"]["extended_tweet"]["entities"]["hashtags"])
        else:
            tw_entities_hashtags.extend(tweet["retweeted_status"]["entities"]["hashtags"])
    except:
        tw_entities_hashtags.extend(tweet["entities"]["hashtags"])
extracted=tweet_extrac(tweet) #extracts_twts.append(extracted)
# mentions_info=[tweet["id_str"],
#                tweet["user"]["screen_name"],
#                tweet["user"]["id_str"],
#                list(set([ent["id_str"] for ent in tw_entities])),
#                list(set([ent["screen_name"] for ent in tw_entities]))]
# out_mentions_cands=[tweet["id_str"]]
# out_mentions_cands.extend([cand_id in mentions_info[2] for cand_id in ids_cands])


In [106]:
tw_entities_hashtags
# extracted
# mentions_info

[{'text': 'DiaDeLaMujer', 'indices': [120, 133]},
 {'text': '8M', 'indices': [134, 137]},
 {'text': 'DiaDeLaMujer', 'indices': [99, 112]},
 {'text': '8M', 'indices': [113, 116]},
 {'text': '8M2020', 'indices': [117, 124]},
 {'text': '8marzo2020', 'indices': [125, 136]}]

In [107]:
tw_entities_mentions

[{'screen_name': 'ChalecosAmarill',
  'name': '©halecos Amarillosᴳᴸᴼᴮᴬᴸ 🔥ʷAͤNͣOͬNͤYˡMͤOᵍUͥSͦⁿ🔥',
  'id': 1071499269880901639,
  'id_str': '1071499269880901639',
  'indices': [3, 19]}]

# Actualizaciones
    Exportar [id tuit, user_name, user_id, id_str, screen_name] for mention in tw_entities_mention <-- es importante eliminar duplicados.
    Exportar [id tuit, user_name, text,] for hashtag in tw_entities_hashtags <-- es importante eliminar duplicados.
    
Ambos a archivos separados independientes